In [1]:
import keras
from keras.layers import Conv2D, Input, Flatten, Dense, Lambda, Reshape
from keras.layers import BatchNormalization
from keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
from tensorflow.python.framework.ops import disable_eager_execution
#disable_eager_execution()

In [3]:
import os
import PIL
import PIL.Image
import pathlib
from sklearn.model_selection import train_test_split
#data_dir = pathlib.Path("C:/Users/Davran/0_Deep_Learning_Project/Implant Inspection/All_bok_div")
#data_dir_train = pathlib.Path("C:/Users/Davran/0_Deep_Learning_Project/Implant Inspection/DATASET_ORIGINAL/train/*/*")
#data_dir_test = pathlib.Path("C:/Users/Davran/0_Deep_Learning_Project/Implant Inspection/DATASET_ORIGINAL/test/*/*")

data_dir = pathlib.Path("C:/Users/Davran/0_Deep_Learning_Project/Implant Inspection")
data_dir_train = pathlib.Path("C:/Users/Davran/0_Deep_Learning_Project/Implant Inspection/All_bok_div/*/*")
data_dir_test = pathlib.Path("C:/Users/Davran/0_Deep_Learning_Project/Implant Inspection/All_bok_div/*/*")

# Loading Dataset

In [4]:
filenames = tf.io.gfile.glob(str(data_dir_train))
#filenames.extend(tf.io.gfile.glob(str(data_dir_test)))

train_filenames, val_filenames = train_test_split(filenames, test_size=0.2)

In [5]:
COUNT_ISOLA = len([filename for filename in train_filenames if "Isola" in filename])
print("Isola images count in training set: " + str(COUNT_ISOLA))

COUNT_LEGACY = len([filename for filename in train_filenames if "Legacy" in filename])
print("Legacy images count in training set: " + str(COUNT_LEGACY))

COUNT_VIPER = len([filename for filename in train_filenames if "Viper" in filename])
print("Viper images count in training set: " + str(COUNT_VIPER))

COUNT_RELINE = len([filename for filename in train_filenames if "Reline" in filename])
print("Reline images count in training set: " + str(COUNT_RELINE))

COUNT_M8M10 = len([filename for filename in train_filenames if "M8M10" in filename])
print("M8M10 images count in training set: " + str(COUNT_M8M10))

COUNT_XIA = len([filename for filename in train_filenames if "Xia" in filename])
print("Xia images count in training set: " + str(COUNT_XIA))

Isola images count in training set: 21
Legacy images count in training set: 74
Viper images count in training set: 111
Reline images count in training set: 43
M8M10 images count in training set: 30
Xia images count in training set: 61


In [ ]:
train_list_ds = tf.data.Dataset.from_tensor_slices(train_filenames)
val_list_ds = tf.data.Dataset.from_tensor_slices(val_filenames)

for f in train_list_ds.take(5):
    print(f.numpy())

In [ ]:
TRAIN_IMG_COUNT = tf.data.experimental.cardinality(train_list_ds).numpy()
print("Training images count: " + str(TRAIN_IMG_COUNT))

VAL_IMG_COUNT = tf.data.experimental.cardinality(val_list_ds).numpy()
print("Validating images count: " + str(VAL_IMG_COUNT))

In [ ]:
CLASS_NAMES = np.array([str(tf.strings.split(item, os.path.sep)[-1].numpy())[2:-1]
                        for item in tf.io.gfile.glob(str(data_dir) + "/All_bok_div/*")])
CLASS_NAMES

# Data Preparation

In [ ]:
def get_label(file_path):
    # convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    if parts[-2] == 'Isola':
        return 1
    elif parts[-2] == 'Viper':
        return 2
    elif parts[-2] == 'M8M10':
        return 3
    elif parts[-2] == 'Reline':
        return 4
    elif parts[-2] == 'Legacy':
        return 5
    else:
        return 6

In [ ]:
def decode_img(img):
  # convert the compressed string
  img_X = tf.image.decode_jpeg(img, channels=1)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img_X = tf.image.convert_image_dtype(img_X, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img_X, IMAGE_SIZE)

In [ ]:
def process_path(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
IMAGE_SIZE = [28, 28]
train_ds = train_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

val_ds = val_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [ ]:
X_train = []
X_test = []
y_train = []
y_test = []
for image, label in train_ds:
    X_train.append(image)
    y_train.append(label)    
X_train = np.array(X_train, 'float32')
y_train = np.array(y_train, 'float32')

#trainX = X_train.copy()
#trainy = y_train.copy()

In [ ]:
for image, label in val_ds:
    X_test.append(image)
    y_test.append(label)
X_test = np.array(X_test, 'float32')
y_test = np.array(y_test, 'float32')

#testX = X_test.copy()
#testy = y_test.copy()

In [ ]:
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

# VAE

## Encoder Part

In [ ]:
import tensorflow

input_data = tensorflow.keras.layers.Input(shape=(28, 28, 1))

encoder = tensorflow.keras.layers.Conv2D(64, (3,3), activation='relu')(input_data)
#encoder = tensorflow.keras.layers.MaxPooling2D((2,2))(encoder)

encoder = tensorflow.keras.layers.Conv2D(64, (3,3), activation='relu')(encoder)
#encoder = tensorflow.keras.layers.MaxPooling2D((2,2))(encoder)

encoder = tensorflow.keras.layers.Conv2D(32, (3,3), activation='relu')(encoder)
#encoder = tensorflow.keras.layers.MaxPooling2D((2,2))(encoder)

encoder = tensorflow.keras.layers.Flatten()(encoder)
encoder = tensorflow.keras.layers.Dense(32)(encoder)
encoder = tensorflow.keras.layers.Dense(32)(encoder)

## Latent Distribution and Sampling

In [ ]:
def sample_latent_features(distribution):
    distribution_mean, distribution_variance = distribution
    batch_size = tensorflow.shape(distribution_variance)[0]
    random = tensorflow.keras.backend.random_normal(shape=(batch_size, tensorflow.shape(distribution_variance)[1]))
    return distribution_mean + tensorflow.exp(0.5 * distribution_variance) * random

In [ ]:
distribution_mean = tensorflow.keras.layers.Dense(64, name='mean')(encoder)
distribution_variance = tensorflow.keras.layers.Dense(64, name='log_variance')(encoder)
latent_encoding = tensorflow.keras.layers.Lambda(sample_latent_features)([distribution_mean, distribution_variance])

In [ ]:
## without hyperparameters

encoder_model = tensorflow.keras.Model(input_data, latent_encoding)
encoder_model.summary()

In [ ]:
#encoder_model = tensorflow.keras.Model(input_data, latent_encoding)
#encoder_model.summary()

## Decoder Part

In [ ]:
decoder_input = tensorflow.keras.layers.Input(shape=(64))
decoder = tensorflow.keras.layers.Dense(21*21*32)(decoder_input)
decoder = tensorflow.keras.layers.Reshape((21, 21, 32))(decoder)
decoder = tensorflow.keras.layers.Conv2DTranspose(32, (3,3), activation='relu')(decoder)
decoder = tensorflow.keras.layers.UpSampling2D((2,2))(decoder)

decoder = tensorflow.keras.layers.Conv2DTranspose(32, (3,3), activation='relu')(decoder)
decoder = tensorflow.keras.layers.UpSampling2D((2,2))(decoder)

decoder = tensorflow.keras.layers.Conv2DTranspose(64, (3,3), activation='relu')(decoder)
#decoder = tensorflow.keras.layers.UpSampling2D((2,2))(decoder)

decoder_output = tensorflow.keras.layers.Conv2DTranspose(1, (3,3), activation='relu')(decoder)

In [ ]:
decoder_model = tensorflow.keras.Model(decoder_input, decoder_output)
decoder_model.summary()

## Combining

In [ ]:
encoded = encoder_model(input_data)
decoded = decoder_model(encoded)

In [ ]:
autoencoder = tensorflow.keras.models.Model(input_data, decoded)

## Loss Function (Reconstruction Loss + KL-loss)

In [ ]:
def get_loss(distribution_mean, distribution_variance):
    
    def get_reconstruction_loss(y_true, y_pred):
        reconstruction_loss = tensorflow.keras.losses.mse(y_true, y_pred)
        reconstruction_loss_batch = tensorflow.reduce_mean(reconstruction_loss)
        return reconstruction_loss_batch
    
    def get_kl_loss(distribution_mean, distribution_variance):
        kl_loss = 1 + distribution_variance - tensorflow.square(distribution_mean) - tensorflow.exp(distribution_variance)
        kl_loss_batch = tensorflow.reduce_mean(kl_loss)
        return kl_loss_batch*(-0.5)
    
    def total_loss(y_true, y_pred):
        reconstruction_loss_batch = get_reconstruction_loss(y_true, y_pred)
        kl_loss_batch = get_kl_loss(distribution_mean, distribution_variance)
        return reconstruction_loss_batch + kl_loss_batch
    
    return total_loss

## Training

In [ ]:
autoencoder.compile(loss=get_loss(distribution_mean, distribution_variance), optimizer='adam')
autoencoder.summary()

In [ ]:
autoencoder.fit(X_train, X_train, epochs=100, batch_size=16, validation_data=(X_test, X_test))

In [ ]:
offset=20
print ("Real Test Images")
# Real Images
plt.figure(figsize = (10, 10))
for i in range(9):
    plt.subplot(330 + 1 + i)
    plt.imshow(X_test[i+offset,:,:, -1], cmap='gray')
plt.show()

In [ ]:
# Reconstructed Images
print ("Reconstructed Images with Variational Autoencoder")
plt.figure(figsize = (10, 10))
for i in range(9):
    plt.subplot(330 + 1 + i)
    output = autoencoder.predict(np.array([X_test[i+offset]]))
    op_image = np.reshape(output[0]*255, (100, 100))
    plt.imshow(op_image, cmap='gray')
plt.show()

## Latent feature clusters

In [ ]:
x = []
y = []
z = []
for i in range(500):
    z.append(testy[i])
    op = encoder_model.predict(np.array([X_test[i]]))
    x.append(op[0][0])
    y.append(op[0][1])

In [ ]:
df = pd.DataFrame()
df['x'] = x
df['y'] = y
df['z'] = ["digit-"+str(k) for k in z]

plt.figure(figsize=(8, 6))
sns.scatterplot(x='x', y='y', hue='z', data=df)
plt.show()

## Image Generation

In [ ]:
generator_model = decoder_model

In [ ]:
x_values = np.linspace(-3, 3, 30)
y_values = np.linspace(-3, 3, 30)

In [ ]:
figure = np.zeros((28 * 30, 28 * 30))
for ix, x in enumerate(x_values):
    for iy, y in enumerate(y_values):
        latent_point = np.array([[x, y]])
        generated_image = generator_model.predict(latent_point)[0]
        figure[ix*28:(ix+1)*28, iy*28:(iy+1)*28,] = generated_image[:,:,-1]
 
plt.figure(figsize=(15, 15))
plt.imshow(figure, cmap='gray', extent=[3,-3,3,-3])
plt.show()